# AdaBoost_Trees1
Cuidado com o Overfitting, fzr cross validation para saber o melhor numero de stumps

Temos várias maneiras de fazer AdaBoost:
- Sempre que fazemos um node, o objetivo é obter o menor erro possível

Por agora, vamos tentar focar no (1), mas tentar colocar args nas funções para também poderem fazer o (2)

In [2]:
import pandas as pd
import numpy as np
import random

In [46]:
df_=pd.read_csv('titanic_train.csv')
df_=df_.drop(['PassengerId','Name','SibSp','Parch','Ticket','Fare','Cabin','Embarked'],axis=1)
survived=df_['Survived']
df_=df_.drop(['Survived'],1)
df_=pd.concat([df_,pd.DataFrame(survived)],sort=False,axis=1)
df_=df_.dropna().reset_index(drop=True)
df_1=df_.head(20)
df_test=df_.tail(50)

In [2]:
import pandas as pd
import numpy as np
import random

print('Amount_of_Say, Weights, ... vão estar na ordem das unidades com 6 casas decimais')
print('também está feito apenas para variáveis 0/1')

def var_values(df,column): #dá nos uma lista dos valores de uma variável 
    #Nota, cada variável está associada a uma coluna
    coluna=list(df.iloc[:,column]) #transforma coluna em lista
    q=[]
    for val in coluna:
        if val not in q:
            q+=[val]
    return q

######################## Algoritmo de ordenação Mergesort
def fusao(u,v):
    res=[]
    i=0
    j=0
    for k in range(len(u)+len(v)):
        if i<len(u) and (j==len(v) or u[i]<v[j]):
            res.append(u[i])
            i=i+1
        else:
            res.append(v[j])
            j=j+1
    return res

def mergesort(w):
    if len(w)<2:
        return w
    else:
        m=len(w)//2
        w1=mergesort(w[:m])
        w2=mergesort(w[m:])
        return fusao(w1,w2)
#  #  #  #   #   #  #   #   #  #  #  #  #   # ALGORITMO DE ORDENAÇÃO MERGESORT



def is_numeric(x): #diz se x é número ou não
    return (isinstance(x,int)) or (isinstance(x,float)) or (isinstance(x,np.int64)) or (isinstance(x,np.float64))

#NOTAAA: Parece q os números do pandas estão em numpy.int64  ou numpy.float64


def string_column(df,column):
    #True se coluna constituída por strings
    q=var_values(df,column)
    l=len(q)
    i=0
    found=False
    while i<l and not(found):
        if not(isinstance(q[i],str)):
            found=True
        i+=1
    return not(found)
    

def is_categorical(df,column): 
    #diz se uma coluna numérica é categórica (sim/não, 0/1) (True)
    #ou se é variável contínua (False)
    if not(string_column(df,column)):
        return len(var_values(df,column))==2
    else:
        print('Esta coluna é de str, não mede esta função')
        return 'Esta coluna é de str, não mede esta função'
    
    
       
def not_categorical_values(df,column):
    #dá as médias dos valores adjacentes para fazer as perguntas
    v1=mergesort(var_values(df,column)) #dá jeito ter estes valores ordenados
    values=[]
    for i in range(len(v1)-1):
        values+=[((v1[i]+v1[i+1])/2)]
    return values 
 
    
    
def ada_depend_counts(df):
    #devolve um dicionário com os valores da variável dependente e as vezes q aparecem
    #É para ser usada no adaboost, pois neste modelo, colocamos os pesos na coluna -1, e as var dep. na -2
    counts={}
    coluna=list(df.iloc[:,-2])
    for val in coluna:
        if val not in counts:
            counts[val]=0
        counts[val]+=1
    return counts

class Question:
    
    def __init__(self,column,value1,value2=None,tipo=0):
        self.column=column
        self.value1=value1
        self.value2=value2
        self.tipo=tipo
        
    def match(self,example):
        #verifica se o example dá True ou False à pergunta
        #example é uma linha/amostra em pd.DataFrame
        val=example.iloc[0][self.column] #como é uma linha, queremos 0
        if is_numeric(val):
            if self.tipo==0:
                return val > self.value1
            else:
                return self.value1<= val <= self.value2
        else:
            return val == self.value1
            
def partition(df, question):
    #divide uma dataframe em valores que que acertam a pergunta e que erram
    true_rows,false_rows=pd.DataFrame(),pd.DataFrame()
    for i in range(len(df.index)):
        l=pd.DataFrame(df.iloc[i,:]).T #linha/amostra em linha
        if question.match(l):
            true_rows=pd.concat([true_rows,l],sort=False)
        else:
            false_rows=pd.concat([false_rows,l],sort=False)
    false_rows=false_rows.reset_index(drop=True)
    true_rows=true_rows.reset_index(drop=True)
    return true_rows,false_rows


def choose_correct(t,f): #escolhe qual das folhas do stump prevê 1 e qual prevê 0
    #devolve boleano, erro menor
    #True se true_rows prever 1
    #False se true_rows prever 0
    #caso 1, true->1  false->0
    #caso 2, true->0  false->1
    #t,f=partition(df,question)
    sizef=len(f.index)
    sizet=len(t.index)
    erro1=0  #caso 1, true->1  false->0
    erro2=0  #caso 2, true->0  false->1
    for i in range(sizet):
        if t.iloc[i][-2]==0:
            erro1+=t.iloc[i][-1]
        else:
            erro2+=t.iloc[i][-1]
    for n in range(sizef):
        if f.iloc[n][-2]==1:
            erro1+=f.iloc[n][-1]
        else:
            erro2+=f.iloc[n][-1]
    #print(erro1,erro2)        
    if erro1>=erro2:
        return False,erro2
    else:
        return True,erro1
    
def Ada_best_column_question(df,column,perguntas=2):
    best_question=None
    boly=None
    Err=1
    T1=None
    F1=None
    if string_column(df,column):
        #print('gone to string')
        values = var_values(df,column)
        for val in values:
            q=Question(column,val)
            t,f =partition(df,q)
            #se a pergunta não divide os dados fazemos SKIP
            if len(t) == 0 or len(f) == 0:
                continue
            b, erro=choose_correct(t,f)
            if erro <= Err:
                boly, Err, best_question = b, erro, q
    else:
        if is_categorical(df,column):
            #print('gone to categorical')
            values = var_values(df,column) 
            assert len(values)==2
            value = (values[0]+values[1])/2
            q=Question(column,value,tipo=0)
            t,f =partition(df,q)
            b, erro=choose_correct(t,f)
            if erro <= Err:
                boly, Err ,best_question = b,erro,q
        else:
            #print('oh shit, here we go again, best_question')
            values=not_categorical_values(df,column)
            #print(values,'oh shit values')
            for i in range(perguntas): #Faz ciclo dos tipos de perguntas (1 ou 2)
                if i==0: #para perguntas tipo 0
                    for val in values:
                        q=Question(column,val,tipo=0)
                        t,f =partition(df,q)
                        #se a pergunta não divide os dados fazemos SKIP
                        if len(t) == 0 or len(f) == 0:
                            continue
                        b, erro=choose_correct(t,f)
                        if erro <= Err:
                            boly, Err, best_question = b, erro, q
                            
                else: #para perguntas tipo 1
                    for i1 in range(1,len(values)-2): #se i1 for para lá de values[-2], não rende (ver caderno)
                        for i2 in range(i1+1,len(values)-1):
                            q=Question(column,values[i1],values[i2],tipo=1)
                            t,f =partition(df,q)
                            #se a pergunta não divide os dados fazemos SKIP
                            if len(t) == 0 or len(f) == 0:
                                continue
                            b, erro=choose_correct(t,f)
                            if erro <= Err:
                                boly, Err, best_question = b, erro, q
    #print(best_question.value1,best_question.value2,best_question.tipo)
    return best_question,boly,Err

def Ada_best_stump(df): #NOTA, esta função apenas serve para ADABOOST, não tem random de colunas
    #ou de amostras, para fazer Random_Forest
    Err=1
    best_question=None
    boly=None
    T1=None
    F1=None
    columns=len(df.columns)
    for i in range(columns-2): #a última coluna são os pesos e a -2 as var. dep
        if len(var_values(df,i))==1:
            continue
        q, b, erro=Ada_best_column_question(df,i)
        if erro <= Err:
            best_question,boly, Err = q, b, erro
    #print(best_question.column,best_question.value1,best_question.value2,best_question.tipo)
    return best_question, boly, Err

def amount_of_say(error):
    if error==0:
        return round((1/2)*np.log(1/(0+10**(-308))),6) #fui testar, e é mínimo que podemos adicionar sem dar erro
    elif error==1:
        return round((1/2)*np.log((0+10**(-323))/1),6) #fui testar, e é mínimo que podemos adicionar sem dar erro
    elif 0<error<1:
        return round((1/2)*np.log((1-error)/error),6)
    else:
        print('Erro no amount_of_say')
        return 'Erro no amount_of_say'







class AdaLeaf:
#as adaLeaf dizem se preveem 0/1  
    def __init__(self,var):
        self.var=var
        
    #Leaf é um dicionário com as var. dep e as vezes q aparecem
    
class Stump:
    def __init__(self,question,leaf1,leaf2,a_of_say):
        self.question=question
        self.true_branch=leaf1
        self.false_branch=leaf2
        self.a_of_say=a_of_say #amount of say
        
    
class AdaForest:
    #floresta de Stumps
    
    def __init__(self):
        self.stumps=[]
        
    def add_stump(self,stump):
        (self.stumps)+=[stump]
        
    def size(self):
        return len(self.stumps)
    
    def get_stump(self,ind):
        return self.stumps[ind]
    
    def merge_adaforests(self,adaforest):
        (self.stumps)+=(adaforest.stumps)
        
   
    
    
def first_weights(df,auto=True,var=1,var_percentage=70):
    size=len(df.index)
    if auto:
        simple=[1]*size
        w1=list(map(lambda x:round(x/size,6),simple))
        weights=pd.DataFrame({'Sample_Weights':w1})
        df=pd.concat([df,weights],sort=False,axis=1)
    else:
        varz=0
        notvarz=0
        for i in range(size):
            if df.iloc[i][-1]==var: #as var dep ainda estão na última coluna
                varz+=1
            else:
                notvarz+=1
        dep=list(df.iloc[:,-1])
        w1=list(map(lambda x: (var_percentage)/(varz*100) if x==var else ((100-var_percentage)/(notvarz*100)),dep))
        weights=pd.DataFrame({'Sample_Weights':w1})
        df=pd.concat([df,weights],sort=False,axis=1)
        
    return df

def right_wrong(df,question,boly):
    rw=[]
    for i in range(len(df.index)):
        l=pd.DataFrame(df.iloc[i,:]).T #linha/amostra em linha
        ind=df.iloc[i][-2]
        if question.match(l): #True
            if ind==1:
                if boly:
                    rw+=['R']
                else:
                    rw+=['W']
            else:
                if boly:
                    rw+=['W']
                else:
                    rw+=['R']
        else:
            if ind==1:
                if boly:
                    rw+=['W']
                else:
                    rw+=['R']
            else:
                if boly:
                    rw+=['R']
                else:
                    rw+=['W']
    return rw
                
    
        
def aux_weights(df,rw,a_say):
    size=len(df.index)
    old_weight=list(df.iloc[:,-1])
    df=df.drop('Sample_Weights',axis=1)
    new_weight=[]
    for i in range(size):
        if rw[i]=='R':
            new_weight+=[old_weight[i]*(np.e**(-a_say))]
        else:
            new_weight+=[old_weight[i]*(np.e**(a_say))]
    return df,new_weight

def new_weights(df,rw,a_say):
    df,new_weights=aux_weights(df,rw,a_say)
    re=sum(new_weights)
    s_weight=list(map(lambda x:round(x/re,6),new_weights))
    Weight=pd.DataFrame({'Sample_Weights':s_weight})
    df=pd.concat([df,Weight],sort=False,axis=1)
    return df
    
    
    
def build_stump(df):
    #função para AdaBoost, não serve para mais nada
    best_question, boly, Err = Ada_best_stump(df)
    rw=right_wrong(df,best_question,boly)
    a_say=amount_of_say(Err)
    if boly:
        a1=AdaLeaf(1)
        a2=AdaLeaf(0)   
    else:
        a1=AdaLeaf(0)
        a2=AdaLeaf(1)
    df=new_weights(df,rw,a_say)
    return Stump(best_question,a1,a2,a_say),df
    
    
def build_AdaForest(df,stumps=100,auto=True,var=1,var_percentage=70):
    ada=AdaForest()
    df=first_weights(df,auto=auto,var=var,var_percentage=var_percentage)
    for i in range(stumps):
        print(i,'stump')
        stump,df=build_stump(df)
        ada.add_stump(stump)
    return ada
    
def Ada_predict_row(row,adaforest,auto=True,var=1,limite=70):
    size=adaforest.size()
    ones=0 #guardando o a_say das previsões de 1
    zeros=0  #guardando o a_say das previsões de 1
    for i in range(size):
        stump=adaforest.get_stump(i)
        q=stump.question
        if q.match(row):
            if stump.true_branch.var==1:
                ones+=stump.a_of_say
            else:
                zeros+=stump.a_of_say
        else:
            if stump.false_branch.var==1:
                ones+=stump.a_of_say
            else:
                zeros+=stump.a_of_say
    if auto:
        if ones>zeros:
            return 1
        else:
            return 0
    else:
        if var==1:
            if round(100*(ones/(zeros+ones)),2)>=limite:
                return 1
            else:
                return 0
        elif var==0:
            if round(100*(zeros/(zeros+ones)),2)>=limite:
                return 0
            else:
                return 1
            
            
def Ada_predict(df,adaforest,auto=True,var=1,limite=70):
    size1=len(df.index)
    predictions=[]
    for i in range(size1):
        row=pd.DataFrame(df.iloc[i,:]).T
        pred=Ada_predict_row(row,adaforest,auto=auto,var=var,limite=limite)
        predictions+=[pred]
    return predictions


# #   #  #  #  # FUNÇÕES PARA A CMAT #  #  #  #   #   #   #   #   #  #  #  #  # #  #  #  #  #  #  #  #  #  # #  #   #
def sensitivity(m,ind):
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    print('O índice começa em 0')
    x=m[ind][ind]
    n=0
    q=[]
    while n<len(m):
        if n==ind:
            n+=1
        else:
            q+=[m[n][ind]]
            n+=1
    y=sum(q)
    return round((x/(x+y))*100,2)

def specificity(m,ind):
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    print('O índice começa em 0')
    n=0
    q=[]
    while n<len(m):
        if n==ind:
            n+=1
        else:
            q+=[m[ind][n]]
            n+=1
    x=sum(q)
    f=[]
    i=0
    while i<len(m):
        c=0
        if i==ind:
            i+=1
        else:
            while c<len(m):
                if c==ind:
                    c+=1
                else:
                    f+=[m[i][c]]
                    c+=1
            i+=1
    y=sum(f)
    #print(q,f)  #verifica se os valores q estamos a escolher são os certos
    return round((y/(x+y))*100,2)

#A percentagem de previsões corretas feitas pelo modelo
#i é a linha da confusion matrix (o outcome que queremos ver a percentagem de previsões corretas)
def pred_percent(m,i):
    #print(m)
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    #print('O índice começa em 0')
    x=m[i][i]
    soma=sum(m[i])
    if soma==0:
        return 'Não previu nada para esta variável'
    else:
        return round((x/soma)*100,2)
#  #  # #  # # #  # # #  # # # #  # ##  # #  # # # # #   # #  #  #  ##   #   # #  #   #   #  #   #    #   #   #   #   #   

def Ada_cmat(df,adaforest,auto=False,var=1,limite=60):
    real=list(df.iloc[:,-1]) #VERIFICAR, mas como adicionamos as colunas dos pesos nas funções, as df original fica igual
    predicted=Ada_predict(df,adaforest,auto=auto,var=var,limite=limite)
    print('predicted',predicted)
    cmat=np.array([[0,0],[0,0]])
    for i in range(len(real)):
        if real[i]==1 and predicted[i]==1:
            cmat[0][0]+=1
        elif real[i]==1 and predicted[i]==0:
            cmat[1][0]+=1
        elif real[i]==0 and predicted[i]==1:
            cmat[0][1]+=1
        elif real[i]==0 and predicted[i]==0:
            cmat[1][1]+=1
        else:
            print('Erro na cmat_forest')
            return 'Erro na cmat_forest'
    return cmat, pred_percent(cmat,1-var)


#  #  #  # #  #  #  #  # Funções para vizualisar a Ada_Forest

def Q_info(adaforest,names=False,df=None):
    #devolve os a_of_say,tipo, coluna e valores de cada pergunta (com o seu índice)
    size= adaforest.size()
    info=[]
    for i in range(size):
        stump=adaforest.get_stump(i)
        if names: #dá nos o nome das colunas em vez do índice
            coluna=df.columns[stump.question.column]
        else:
            coluna=stump.question.column
        info+=[[i,stump.a_of_say,coluna,stump.question.tipo,stump.question.value1,stump.question.value2]]
    return info

def used_columns(adaforest,names=False,df=None):
    #dá um dicionário com as vezes que cada coluna foi usada
    dic={}
    size= adaforest.size()
    for i in range(size):
        stump=adaforest.get_stump(i)
        if names: #dá nos o nome das colunas em vez do índice
            coluna=df.columns[stump.question.column]
        else:
            coluna=stump.question.column
        if coluna not in dic:
            dic[coluna]=0
        dic[coluna]+=1
    return dic

Amount_of_Say, Weights, ... vão estar na ordem das unidades com 6 casas decimais
também está feito apenas para variáveis 0/1


In [ ]:
class AdaForest:
    #floresta de Stumps
    
    def __init__(self):
        self.stumps=[]
        
    def add_stump(self,stump):
        (self.stumps)+=[stump]
        
    def size(self):
        return len(self.stumps)
    
    def get_stump(self,ind):
        return self.stumps[ind]
    
    def merge_adaforests(self,adaforest):
        (self.stumps)+=(adaforest.stumps)
        

In [23]:
%time Ada=build_AdaForest(df_1,stumps=50)

Wall time: 4min 38s


In [32]:
row1=pd.DataFrame(df_1.iloc[2,:]).T
row1

,Pclass,Sex,Age,Survived
2,3,female,26,1


In [53]:
Ada_cmat(df_test,Ada,auto=False,var=1,limite=64)

predicted [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]


(array([[ 6,  0],
        [14, 30]]), 100.0)

In [45]:
df_test

,Pclass,Sex,Age,Survived
709,3,female,39.0,0
710,2,male,27.0,0
711,1,female,19.0,1
712,1,male,26.0,1
713,3,male,32.0,0


In [87]:
oioi=first_weights(df_1,auto=False,var=0,var_percentage=80)
oioi.drop('Sample_Weights',axis=1)

,Pclass,Sex,Age,Survived
0,3,male,22.0,0
1,1,female,38.0,1
2,3,female,26.0,1
3,1,female,35.0,1
4,3,male,35.0,0
5,1,male,54.0,0
6,3,male,2.0,0
7,3,female,27.0,1
8,2,female,14.0,1
9,3,female,4.0,1


##### Sample Weight tem de ser incorporado na df (última coluna)


In [7]:
df_1=first_sample_weight(df_1)

In [8]:
weighted_gini(df_1)

[10, 0.49999999999999994] zero
[10, 0.49999999999999994] one
p0,p1 0.5 0.5


0.5

_________________________

In [10]:
import pandas as pd
import numpy as np
import random

print('Este módulo usa pandas.Dataframe')
print('E está feito para apenas prever variáveis categóricas 0/1')

def var_values(df,column): #dá nos uma lista dos valores de uma variável 
    #Nota, cada variável está associada a uma coluna
    coluna=list(df.iloc[:,column]) #transforma coluna em lista
    q=[]
    for val in coluna:
        if val not in q:
            q+=[val]
    return q

######################## Algoritmo de ordenação Mergesort
def fusao(u,v):
    res=[]
    i=0
    j=0
    for k in range(len(u)+len(v)):
        if i<len(u) and (j==len(v) or u[i]<v[j]):
            res.append(u[i])
            i=i+1
        else:
            res.append(v[j])
            j=j+1
    return res

def mergesort(w):
    if len(w)<2:
        return w
    else:
        m=len(w)//2
        w1=mergesort(w[:m])
        w2=mergesort(w[m:])
        return fusao(w1,w2)
#  #  #  #   #   #  #   #   #  #  #  #  #   # ALGORITMO DE ORDENAÇÃO MERGESORT



def is_numeric(x): #diz se x é número ou não
    return (isinstance(x,int)) or (isinstance(x,float)) or (isinstance(x,np.int64)) or (isinstance(x,np.float64))

#NOTAAA: Parece q os números do pandas estão em numpy.int64  ou numpy.float64


def string_column(df,column):
    #True se coluna constituída por strings
    q=var_values(df,column)
    l=len(q)
    i=0
    found=False
    while i<l and not(found):
        if not(isinstance(q[i],str)):
            found=True
        i+=1
    return not(found)
    

def is_categorical(df,column): 
    #diz se uma coluna numérica é categórica (sim/não, 0/1) (True)
    #ou se é variável contínua (False)
    if not(string_column(df,column)):
        return len(var_values(df,column))==2
    else:
        print('Esta coluna é de str, não mede esta função')
        return 'Esta coluna é de str, não mede esta função'
    
    
       
def not_categorical_values(df,column):
    #dá as médias dos valores adjacentes para fazer as perguntas
    v1=mergesort(var_values(df,column)) #dá jeito ter estes valores ordenados
    values=[]
    for i in range(len(v1)-1):
        values+=[((v1[i]+v1[i+1])/2)]
    return values 
 
    
    
def ada_depend_counts(df):
    #devolve um dicionário com os valores da variável dependente e as vezes q aparecem
    #É para ser usada no adaboost, pois neste modelo, colocamos os pesos na coluna -1, e as var dep. na -2
    counts={}
    coluna=list(df.iloc[:,-2])
    for val in coluna:
        if val not in counts:
            counts[val]=0
        counts[val]+=1
    return counts

class Question:
    
    def __init__(self,column,value1,value2=None,tipo=0):
        self.column=column
        self.value1=value1
        self.value2=value2
        self.tipo=tipo
        
    def match(self,example):
        #verifica se o example dá True ou False à pergunta
        #example é uma linha/amostra em pd.DataFrame
        val=example.iloc[0][self.column] #como é uma linha, queremos 0
        if is_numeric(val):
            if self.tipo==0:
                return val > self.value1
            else:
                return self.value1<= val <= self.value2
#as perguntas tipo 1 podem dar muitoooooooo jeito
        else:
            #se for uma string, não dá para ser maior/menor q uma string
            return val == self.value1
        
def partition(df, question):
    #divide uma dataframe em valores que que acertam a pergunta e que erram
    true_rows,false_rows=pd.DataFrame(),pd.DataFrame()
    for i in range(len(df.index)):
        l=pd.DataFrame(df.iloc[i,:]).T #linha/amostra em linha
        if question.match(l):
            true_rows=pd.concat([true_rows,l],sort=False)
        else:
            false_rows=pd.concat([false_rows,l],sort=False)
    false_rows=false_rows.reset_index(drop=True)
    true_rows=true_rows.reset_index(drop=True)
    return true_rows,false_rows


def ada_info_gain(true,false,current_impurity):
    #true/false são as df q resultam de fazer uma pergunta (definida previamente)
    #current_impurity é o gini da df sem ser dividida por uma pergunta
    p=(len(true)/(len(true)+len(false)))
    return current_impurity - ((p*weighted_gini(true))+((1-p)*wieghted_gini(false)))

def ada_best_column_question(df,column,perguntas=2): #de uma coluna devolve a melhor pergunta e o seu info_gain
    current_impurity=weighted_gini(df)
    best_gain=0
    best_question=None
    
    if string_column(df,column):
        #print('gone to string')
        values = var_values(df,column)
        for val in values:
            q=Question(column,val)
            t,f =partition(df,q)
            #se a pergunta não divide os dados fazemos SKIP
            if len(t) == 0 or len(f) == 0:
                continue
            gain = info_gain(t,f,current_impurity)
            if gain >= best_gain:
                best_gain, best_question = gain, q
    else:
        if is_categorical(df,column):
            #print('gone to categorical')
            values = var_values(df,column) 
            assert len(values)==2
            value = (values[0]+values[1])/2
            q=Question(column,value,tipo=0)
            t,f =partition(df,q)
            gain = info_gain(t,f,current_impurity)
            if gain >= best_gain:
                best_gain, best_question = gain, q
        else:
            #print('oh shit, here we go again, best_question')
            values=not_categorical_values(df,column)
            #print(values,'oh shit values')
            for i in range(perguntas): #Faz ciclo dos tipos de perguntas (1 ou 2)
                if i==0: #para perguntas tipo 0
                    for val in values:
                        q=Question(column,val,tipo=0)
                        t,f =partition(df,q)
                        #se a pergunta não divide os dados fazemos SKIP
                        if len(t) == 0 or len(f) == 0:
                            continue
                        gain = info_gain(t,f,current_impurity)
                        if gain >= best_gain:
                            best_gain, best_question = gain, q
                            
                else: #para perguntas tipo 1
                    for i1 in range(1,len(values)-2): #se i1 for para lá de values[-2], não rende (ver caderno)
                        for i2 in range(i1+1,len(values)-1):
                            q=Question(column,values[i1],values[i2],tipo=1)
                            t,f =partition(df,q)
                            #se a pergunta não divide os dados fazemos SKIP
                            if len(t) == 0 or len(f) == 0:
                                continue
                            gain = info_gain(t,f,current_impurity)
                            if gain >= best_gain:
                                best_gain, best_question = gain, q
    #print(best_question.value1,best_question.value2,best_question.tipo)
    return best_question,best_gain

#  #  #   #   #   #  # Função Best_Node (também serve para as Random Forests)
def ada_best_node(df,forest=False,auto=True,columns_percentage=50): #melhor pergunta de todas as colunas
    columns=len(df.columns)
    best_question=None
    best_gain=0
    if not(forest): #se não quisermos usar esta função para o modelo Random Forests
        #print(columns-1,'columns, best_node')
        for i in range(columns-2): #a última coluna são os pesos e a -2 as var. dep
            #print(i,'column,best_node')
            if len(var_values(df,i))==1:
               # print('opa, afinal tava mal --------------------')
                continue
            q, gain=best_column_question(df,i)
            if gain >= best_gain:
                best_gain, best_question = gain, q
        #print(best_question.column,best_question.value1,best_question.value2,best_question.tipo)
        return best_question, best_gain
    
    else: # se quisermos usar esta função para o modelo Random_Forests
        
        if auto: #em modo automático usa a raíz quadrada das colunas
            stop=round(np.sqrt(columns)) 
        else: #se não tiver em auto, escolhemos a percentagem de perguntas a analisar
            stop=round((columns*(columns_percentage))/100)
            
        ind=[] #lista dos índices das colunas q vamos usar
        i=0
        done=False
        while i<columns and not(done):
            col=random.randrange(columns)
            if col==(columns-1) or col==(columns-2): #não queremos os pesos nem as var dep
                i+=1
                continue
            ind+=[col]      #este ciclo encontra as colunas aleatórias 
            if len(ind)==stop:                    # que vamos utilizar
                done=True
            i+=1
        for n in ind:
            q, gain=best_column_question(df,n)
            if gain >= best_gain:
                best_gain, best_question = gain, q
        #print(best_question.column,best_question.value1,best_question.value2,best_question.tipo)
        return best_question, best_gain
        
##  #  #  #   #  #   #   #  #   #   #   #   #   #   #   #   #   #   #  #  #  

def proba(df):
    #calcula a probabilidade dos valores de um dicionário (em percentagem, para evitar float errors)
    #É necessário para as Leafs
    ind=len(df.index)
    dicti=depend_counts(df)
    new={0:0,1:0}  # este módulo de3 decision trees está feito para var dep de 0/1 APENAS
    for var in dicti:
        prob=round(100*(dicti[var]/ind),1)
        new[var]=prob
    return new


class AdaLeaf:
#as adaLeaf apenas dizem se   
    def __init__(self,var):
        self.var=var
        
    #Leaf é um dicionário com as var. dep e as vezes q aparecem
    
class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question=question
        self.true_branch=true_branch
        self.false_branch=false_branch

def build_stump(df,limit=1,forest=False,auto=True,columns_percentage=50,count=0):
    #função recursiva que faz árvore (very nice)
    #limit: 2^limit (2**limit) é o número de leafs máximas que queremos
    #limit=None, significa que não tem número máximo e constrói a árvore sem limites
    #count é um 'counter' necessário para fazer funcionar o limit
    #mas como a função é recursiva, ele tem de ser carregado de recursão em recursão
    #como argumento. Não modificar o count qnd chamamos a função é 0 e sempre será
    
    
    if count==limit:
        return AdaLeaf(df)
    q,gain=ada_best_node(df,forest,auto,columns_percentage)
    if gain == 0:
        return AdaLeaf(df)
    t, f =partition(df, q)
    count+=1
    true_branch=build_stump(t,limit=limit,forest=forest,auto=auto,columns_percentage=columns_percentage,count=count)
    false_branch=build_stump(f,limit=limit,forest=forest,auto=auto,columns_percentage=columns_percentage,count=count)
    return Decision_Node(q,true_branch,false_branch)


def ada_classify_row(df,node,probability=True):
    #sevre para chegarmos as folhas
    #só serve para uma linha/amostra de dados serve 
    if isinstance(node,AdaLeaf):
        if probability:
            return node.probability
        else:
            return node.predictions
    if node.question.match(df):
        return classify_row(df,node.true_branch,probability)
    else:
        return classify_row(df,node.false_branch,probability)
    
def test_stump(df,tree,var=1,limite=60):
    #testa uma árvore, sabendo que a var só é considerada 1 se probabilidade>=limite
    #devolve uma lista com os valores q previu

    limit=len(df.index)
    predicted=[] #valores previstos vão estar em lista
    for i in range(limit):
        row=pd.DataFrame(df.iloc[i,:]).T
        d_proba=ada_classify_row(row,tree)
        if d_proba[var]>=60:
            predicted+=[var]
        else:
            predicted+=[1-var] #se var=0,1 e se var=1,0
    return predicted

#  #   #  #  #  #  #  #  # #  # FUNÇÕES PARA A CONFUSION MATRIX
#m é a confusion matrix, ind é o indíce da variável que queremos testar
#ind começa em 0
def sensitivity(m,ind):
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    print('O índice começa em 0')
    x=m[ind][ind]
    n=0
    q=[]
    while n<len(m):
        if n==ind:
            n+=1
        else:
            q+=[m[n][ind]]
            n+=1
    y=sum(q)
    return round((x/(x+y))*100,2)

def specificity(m,ind):
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    print('O índice começa em 0')
    n=0
    q=[]
    while n<len(m):
        if n==ind:
            n+=1
        else:
            q+=[m[ind][n]]
            n+=1
    x=sum(q)
    f=[]
    i=0
    while i<len(m):
        c=0
        if i==ind:
            i+=1
        else:
            while c<len(m):
                if c==ind:
                    c+=1
                else:
                    f+=[m[i][c]]
                    c+=1
            i+=1
    y=sum(f)
    #print(q,f)  #verifica se os valores q estamos a escolher são os certos
    return round((y/(x+y))*100,2)

#A percentagem de previsões corretas feitas pelo modelo
#i é a linha da confusion matrix (o outcome que queremos ver a percentagem de previsões corretas)
def pred_percent(m,i):
    #print(m)
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    #print('O índice começa em 0')
    x=m[i][i]
    soma=sum(m[i])
    if soma==0:
        return 'Não previu nada para esta variável'
    else:
        return round((x/soma)*100,2)

## # # # # #  #  #  #   #   #  #   #   #  #  #  #  #  #  #  #  # #
 
def cmat_stump(df,tree,var=1,limite=60):
    y_real=list(df.iloc[:,-2]) #lembrar q a variável dep está na penúltima coluna no AdaBOOst
    y_predicted=test_stump(df,tree,var,limite)
    cmat=np.array([[0,0],[0,0]])
    for i in range(len(y_real)):
        if y_real[i]==1 and y_predicted[i]==1:
            cmat[0][0]+=1
        elif y_real[i]==1 and y_predicted[i]==0:
            cmat[1][0]+=1
        elif y_real[i]==0 and y_predicted[i]==1:
            cmat[0][1]+=1
        elif y_real[i]==0 and y_predicted[i]==0:
            cmat[1][1]+=1
        else:
            print('Erro na cmat_tree')
            return 'Erro na cmat_tree'
    return cmat, pred_percent(cmat,1-var)
#Porquê '1-var'?
#lembrar q o 2o parametro do pred_percent indica a posição da variável na matriz
#como 1 está na posição 0 e 0 está na posição 1
#a posição é gerada corretamente em função da variável (0/1) 


#######################################################################
#######################################################################
#######################################################################
#######################################################################
#######################################################################
#######################################################################

class Forest:
    def __init__(self):
        self.trees=[]
        
    def add_tree(self,tree):
        (self.trees)+=[tree]
        
    def size(self):
        return len(self.trees)
    
    def get_tree(self,ind):
        return self.trees[ind]
    
    def merge_forests(self,forest):
        (self.trees)+=(forest.trees)
        
class AdaForest:
    
    def __init__(self):
        self.stumps=[]
        
    def add_stump(self,say,stump):
        (self.stumps)+=[[say,stump]]
        
    def size(self):
        return len(self.stumps)
    
    def get_stump(ind):
        return self.stumps[ind]
    
    def merge_adaforests(self,adaforest):
        (self.stumps)+=(adaforest.stumps)
    
    
    
def random_df(df): #cria a tal Bootstrapped dataset 
    size=len(df.index)
    done=False
    new_df=pd.DataFrame()
    i=0
    while i<=(size+2) and not done: #size+2 é só pq sim
        ind=random.randrange(size)
        ds=pd.DataFrame(df.iloc[ind,:]).T
        new_df=pd.concat([new_df,ds],sort=False)
        if len(new_df.index)==size:
            done=True
    return new_df.reset_index(drop=True)
        

def build_forest(df,trees=1000,limit=None,forest=True,auto=True,columns_percentage=50,bootstrap=False,count=0):
    #limit e count servem para a função build_stump
    #bootstrap diz se cada vez que fazemos uma árvore fazemos random da dataframe ou não (T/F)
    F=Forest()
    for i in range(trees):
        if not(bootstrap):
            print('i',i)
            new_tree= build_stump(df,limit=limit,forest=forest,auto=auto,columns_percentage=columns_percentage,count=count)
            F.add_tree(new_tree)
        else:
            print('i',i)
            new_df=random_df(df)
            new_tree= build_stump(new_df,limit=limit,forest=forest,auto=auto,columns_percentage=columns_percentage,count=count)
            F.add_tree(new_tree)
    return F

def forest_classify(df,forest):
    #devolve lista de dicionários em que cada dict refer-se a uma amostra/linha
    #e os valores de cada dict referem-se as previsões de cada árvore em 
    #relação a essa amostra
    rows=len(df.index)
    bigout=[]
    for i in range(rows):
        output={0:0,1:0}
        row=pd.DataFrame(df.iloc[i,:]).T
        trees=forest.size()
        for n in range(trees):
            tree=forest.get_tree(n)
            val=test_tree(row,tree)[0]
            output[val]+=1
        bigout+=[output]
    return bigout

def forest_predictions(df, forest, var=1, limite=60):
    #devolve uma lista com as previsões da floresta
    #impondo um limite (percentagem) a uma variável
    #(só dizemos que se trata dessa variável se a percentagem de probabilidade)
    #for igual ou superior
    final=[]
    classi=forest_classify(df,forest)
    #print(classi)
    for i in classi:
        prob=round((i[var]/(forest.size()))*100)
        if prob>=limite:
            final+=[var]
        else:
            final+=[1-var]
    return final

def cmat_forest(df,forest,var=1,limite=60):
    real=list(df.iloc[:,-1])
    predicted=forest_predictions(df,forest,var,limite)
    print('predicted',predicted)
    cmat=np.array([[0,0],[0,0]])
    for i in range(len(real)):
        if real[i]==1 and predicted[i]==1:
            cmat[0][0]+=1
        elif real[i]==1 and predicted[i]==0:
            cmat[1][0]+=1
        elif real[i]==0 and predicted[i]==1:
            cmat[0][1]+=1
        elif real[i]==0 and predicted[i]==0:
            cmat[1][1]+=1
        else:
            print('Erro na cmat_forest')
            return 'Erro na cmat_forest'
    return cmat, pred_percent(cmat,1-var) 

Este módulo usa pandas.Dataframe
E está feito para apenas prever variáveis categóricas 0/1


In [ ]:
1+1

In [12]:
i=0
while i<20:
    if i<5:
        i+=1
        continue 
    print(i)
    i+=1

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
